In [2]:
import torch

# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

# 1️⃣ Carregar o datasets balanceado já limpo
df = pd.read_csv("../../data/classify_food/meal_detection_balanced.csv")

# 2️⃣ Converter para Dataset Hugging Face
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)

In [4]:

# 3️⃣ Tokenizar com xlm-roberta-base
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize, batched=True)

# 4️⃣ Modelo binário
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

model.to(device)

# 5️⃣ Métricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

Map:   0%|          | 0/19481 [00:00<?, ? examples/s]

Map:   0%|          | 0/4871 [00:00<?, ? examples/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:

# 6️⃣ Treino
training_args = TrainingArguments(
    output_dir="./meal_classifier_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    greater_is_better=False,
    metric_for_best_model="f1",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# 7️⃣ Exportar modelo
trainer.save_model("../models/meal_classifier_model")
tokenizer.save_pretrained("../models/meal_classifier_model")